In [ ]:
# ========================
# 1️⃣ Configuración para Google Colab
# ========================

# Instalar dependencias si es necesario
!pip install keras-tuner

import os
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import random
from google.colab import drive

# ========================
# 2️⃣ Montar Google Drive y cargar archivos
# ========================

drive.mount('/content/drive')

dataset_path = "/content/drive/My Drive/Parcial 1"

train_images = np.load(f"{dataset_path}/kmnist-train-imgs.npz")["arr_0"]
train_labels = np.load(f"{dataset_path}/kmnist-train-labels.npz")["arr_0"]
test_images = np.load(f"{dataset_path}/kmnist-test-imgs.npz")["arr_0"]
test_labels = np.load(f"{dataset_path}/kmnist-test-labels.npz")["arr_0"]

# Normalización
train_images = train_images.astype("float32") / 255.0
test_images = test_images.astype("float32") / 255.0

# Convertir etiquetas a one-hot encoding
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# División en Entrenamiento (85%) y Validación (15%)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.15, random_state=42, shuffle=True
)

# ========================
# 3️⃣ Data Augmentation
# ========================

train_images_reshaped = train_images.reshape(-1, 28, 28, 1)
val_images_reshaped   = val_images.reshape(-1, 28, 28, 1)

datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.10,
    width_shift_range=0.1,
    height_shift_range=0.1
)

# ========================
# 4️⃣ Modelo MLP con Hiperparámetros
# ========================

def build_model(hp):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28, 1)))
    n_neuronas_1 = hp.Choice('n_neuronas_1', [256, 384, 512])
    model.add(Dense(n_neuronas_1, activation='relu', kernel_regularizer=l2(1e-5)))
    model.add(BatchNormalization())
    dropout_1 = hp.Choice('dropout_1', [0.3, 0.4, 0.5])
    model.add(Dropout(dropout_1))
    n_neuronas_2 = hp.Choice('n_neuronas_2', [128, 256, 384])
    model.add(Dense(n_neuronas_2, activation='relu', kernel_regularizer=l2(1e-5)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(10, activation='softmax'))
    lr = 5e-4
    model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ========================
# 5️⃣ Búsqueda de hiperparámetros con Keras Tuner
# ========================

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=9,
    executions_per_trial=1,
    project_name='kmnist_hyperparam_tuning'
)

# ========================
# 6️⃣ Entrenamiento del modelo con los mejores hiperparámetros
# ========================

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

tuner.search(
    datagen.flow(train_images_reshaped, train_labels, batch_size=64),
    validation_data=(val_images_reshaped, val_labels),
    epochs=10,
    callbacks=[early_stopping, reduce_lr]
)

best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
final_model = tuner.hypermodel.build(best_hp)

history = final_model.fit(
    datagen.flow(train_images_reshaped, train_labels, batch_size=64),
    validation_data=(val_images_reshaped, val_labels),
    epochs=20,
    callbacks=[early_stopping, reduce_lr]
)

# ========================
# 7️⃣ Evaluación y visualización de resultados
# ========================

test_images_reshaped = test_images.reshape(-1, 28, 28, 1)
test_loss, test_acc = final_model.evaluate(test_images_reshaped, test_labels)
print(f"\n✅ Accuracy Final en Test: {test_acc:.4f}")

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.xlabel('Épocas')
plt.ylabel('Accuracy')
plt.title('Precisión del Modelo')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.title('Pérdida del Modelo')
plt.legend()
plt.show()

# ========================
# 8️⃣ Matriz de Confusión y Reporte de Clasificación
# ========================

y_pred = final_model.predict(test_images_reshaped)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(test_labels, axis=1)

conf_matrix = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues")
plt.title("Matriz de Confusión")
plt.xlabel("Predicho")
plt.ylabel("Verdadero")
plt.show()

print("\n📌 Reporte de Clasificación:\n")
print(classification_report(y_true, y_pred_classes))


Trial 2 Complete [00h 02m 48s]
val_accuracy: 0.9617778062820435

Best val_accuracy So Far: 0.9617778062820435
Total elapsed time: 00h 05m 34s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
384               |512               |n_neuronas_1
0.4               |0.3               |dropout_1
128               |384               |n_neuronas_2

Epoch 1/10
797/797 ━━━━━━━━━━━━━━━━━━━━ 16s 19ms/step - accuracy: 0.5566 - loss: 1.3927 - val_accuracy: 0.8904 - val_loss: 0.3864 - learning_rate: 5.0000e-04
Epoch 2/10
797/797 ━━━━━━━━━━━━━━━━━━━━ 20s 18ms/step - accuracy: 0.7557 - loss: 0.7608 - val_accuracy: 0.9138 - val_loss: 0.3081 - learning_rate: 5.0000e-04
Epoch 3/10
427/797 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.7920 - loss: 0.6617